# Prepares the dataset in the format needed for training the decision trees

Data has been cleaned and aggregated in R (Alyssa_Script_modified_HSA_areas.R)

In [27]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef
from num2words import num2words
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, RepeatedStratifiedKFold
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score
import word2number
from word2number import w2n
from sklearn.tree import DecisionTreeClassifier
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay
import random
from matplotlib.patches import Polygon

from Functions import prep_training_test_data_period, prep_training_test_data, calculate_metrics,cross_validation_leave_geo_out, prep_training_test_data_shifted, add_labels_to_subplots, LOOCV_by_HSA_dataset, save_in_HSA_dictionary, pivot_data_by_HSA, merge_and_rename_data, add_changes_by_week, create_column_names, create_collated_weekly_data
hfont = {'fontname':'Helvetica'}
palette = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854', '#e5c494']

In [49]:
HSA_weekly_data_updated = pd.read_csv("/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates_with_state_fips.csv")


/var/folders/d4/chxwf4hs5kq7ttsp56s64z65mjk3qj/T/ipykernel_14777/863361005.py:1: DtypeWarning: Columns (43,45,46,48,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  HSA_weekly_data_updated = pd.read_csv("/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates_with_state_fips.csv")


# Initial tidy
- rename columns for brevity
- add the metric of interest ("beds_over_15_per_100k" = indicator variable when beds occupied by COVID-19 patients exceed 15 per 100k)
- rename weeks such that the first date of the dataset is week one, etc. 

In [50]:
HSA_weekly_data_updated.rename(columns={'health_service_area_number': 'HSA_ID'}, inplace=True)
HSA_weekly_data_updated['beds_over_15_100k'] = (HSA_weekly_data_updated['beds_weekly'] > 15)*1
#HSA_weekly_data_updated_features = HSA_weekly_data_updated.dropna(subset=['admits_weekly', 'icu_weekly', 'beds_weekly', 'perc_covid'])
HSA_weekly_data_updated_features = HSA_weekly_data_updated
for i, week in enumerate(HSA_weekly_data_updated_features['date'].unique()):
    HSA_weekly_data_updated_features.loc[HSA_weekly_data_updated_features['date'] == week, 'week'] = i

# Get features of interest and pivot the data such that each column is a HSA 



In [51]:
## pivot 
data_by_HSA__cases = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'cases_weekly')
data_by_HSA__deaths = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'deaths_weekly')
data_by_HSA__admissions = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'admits_weekly')
data_by_HSA__icu = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'icu_weekly')
data_by_HSA__beds = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'beds_weekly')
data_by_HSA__percent_beds = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'perc_covid')
data_by_HSA__over_15_100k = pivot_data_by_HSA(HSA_weekly_data_updated_features, 'week', 'HSA_ID', 'beds_over_15_100k')

## merge 
data_by_HSA__cases_deaths = merge_and_rename_data(data_by_HSA__cases, data_by_HSA__deaths,'week','cases', 'deaths')
data_by_HSA__admits_icu_weekly = merge_and_rename_data(data_by_HSA__admissions, data_by_HSA__icu,'week','admits', 'icu')
data_by_HSA__beds_perc_weekly = merge_and_rename_data(data_by_HSA__beds, data_by_HSA__percent_beds,'week','beds', 'perc_covid')
data_by_HSA__cases_deaths_admits_icu = pd.merge(data_by_HSA__cases_deaths, data_by_HSA__admits_icu_weekly, on='week')
data_by_HSA__cases_deaths_admits_icu_beds = pd.merge(data_by_HSA__cases_deaths_admits_icu, data_by_HSA__beds_perc_weekly, on='week')

## add outcome variable 
old_column_names = data_by_HSA__over_15_100k.columns
new_column_names = [str(col) + '_beds_over_15_100k' for col in old_column_names]
new_column_names = dict(zip(old_column_names, new_column_names))
data_by_HSA__over_15_100k.rename(columns=new_column_names, inplace=True)
data_by_HSA__cases_deaths_admits_icu_beds = pd.merge(data_by_HSA__cases_deaths_admits_icu_beds, data_by_HSA__over_15_100k, on='week')

data_by_HSA__cases_deaths_admits_icu_beds = data_by_HSA__cases_deaths_admits_icu_beds.reset_index()
data_by_HSA__cases_deaths_admits_icu_beds.columns = data_by_HSA__cases_deaths_admits_icu_beds.columns.str.replace(',', '')


# Add changes by week for each continuous feature 

In [52]:
all_HSA_ID_weekly_data = add_changes_by_week(data_by_HSA__cases_deaths_admits_icu_beds, "beds_over_15_100k")

/Users/rem76/Documents/COVID_projections/COVID_forecasting/Functions.py:366: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weekly_data_frame.insert(column_index + 1, new_column_name, diff)
/Users/rem76/Documents/COVID_projections/COVID_forecasting/Functions.py:366: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  weekly_data_frame.insert(column_index + 1, new_column_name, diff)
/Users/rem76/Documents/COVID_projections/COVID_forecasting/Functions.py:366: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

# Change the format such that each week is a column, and each row is a HSA 
- create the column names so the format will be "week_2_cases"

In [56]:
categories_for_subsetting = [ 'cases', 'deaths', 'admits', 'icu', 'beds', 'perc_covid',  'cases_delta', 'deaths_delta', 'admits_delta', 'icu_delta', 'beds_delta', 'perc_covid_delta','beds_over_15_100k']
num_of_weeks = len(all_HSA_ID_weekly_data)
column_names = create_column_names(categories_for_subsetting, num_of_weeks)

In [57]:
all_HSA_ID_weekly_data = create_collated_weekly_data(all_HSA_ID_weekly_data, HSA_weekly_data_updated, categories_for_subsetting, 'HSA_ID', column_names)

# Add weights based on population size (calculated in R file)

In [58]:
weights_df = HSA_weekly_data_updated[HSA_weekly_data_updated['HSA_ID'].isin(all_HSA_ID_weekly_data['HSA_ID'])][['HSA_ID','weight']]
weights_df = weights_df.rename(columns = {'HSA_ID': 'HSA_ID', 'weight':'weight'})
#weights_df = weights_df.drop_duplicates()
weights_df['weight'].unique()
all_HSA_ID_weekly_data = all_HSA_ID_weekly_data.join(weights_df['weight'])

# Write CSV

In [60]:
all_HSA_ID_weekly_data.to_csv("/Users/rem76/Documents/COVID_projections/hsa_time_data_all_dates_weekly_incl_NA.csv", index=False)